In [3]:
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from LVQ.lvq import LVQ3
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [4]:
X_l = np.load("../data/processed/data.npy")
y_l = np.load("../data/processed/label.npy")
X_train, X_test, y_train, y_test = tts(X_l, y_l, test_size=0.20, random_state=15)
output_size = np.unique(y_l).shape[0]
input_size = X_train.shape[1]

In [5]:
sfs = LVQ3(input_size=input_size, output_size=output_size, epoch=3)
sfs.fit(X_train, y_train)
res = sfs.predict(X_test)
print(classification_report(res, y_test, zero_division=1))

              precision    recall  f1-score   support

           0       1.00      0.35      0.52     65819
           1       0.00      0.28      0.00        18
           2       0.00      1.00      0.00         0

    accuracy                           0.35     65837
   macro avg       0.33      0.54      0.17     65837
weighted avg       1.00      0.35      0.52     65837


In [6]:
sfs = LVQ3(input_size=input_size, output_size=output_size, epoch=3, dMethod="cosine")
sfs.fit(X_train, y_train)
res = sfs.predict(X_test)
print(classification_report(res, y_test, zero_division=1))

              precision    recall  f1-score   support

           0       0.66      0.78      0.71     19673
           1       0.25      0.44      0.32     11920
           2       0.77      0.48      0.59     34244

    accuracy                           0.56     65837
   macro avg       0.56      0.57      0.54     65837
weighted avg       0.64      0.56      0.58     65837


In [3]:
# Learning 0.45, 0.35, 0.25
# Epoch: 5, 10, 15
# dMethod: "cosine", "euclidean"
# epsilon: 0.2, 0.3, 0.4
# Beta: 0.25, 0.3, 0.35
# decay: 0.05, 0.15, 0.25


In [4]:
param = {
    "lr": [0.25, 0.35],
    "Epoch": [2, 3],
    "epsilon": [0.25, 0.35],
    "beta": [0.25, 0.3],
    "decay": [0.15, 0.25],
    "dMethod": ["cosine", "euclidean"]
}

In [5]:
result = {}
tot_comb = 2 * 2 * 2 * 2 * 2 * 2

pbar = tqdm(desc="Grid searching", total=tot_comb)
Combination = 1
for dMethod in param["dMethod"]:
    for lr in param["lr"]:
        for epoch in param["Epoch"]:
            for eps in param["epsilon"]:
                for beta in param["beta"]:
                    for decay in param["decay"]:
                        # dMethod, lr, epoch, epsilon, beta, decay, akurasi
                        params = [dMethod, lr, epoch, eps, beta, decay]
                        grid = LVQ3(input_size=input_size, output_size=output_size, dMethod=dMethod, alpha=lr, epoch=epoch,
                                    epsilon=eps, beta=beta, decay=decay, random_state=12345)
                        grid.fit(X_train, y_train)
                        g_res = grid.predict(X_test)
                        params.append(accuracy_score(g_res, y_test))
                        t = "Combination_" + str(Combination)
                        result[t] = params
                        Combination += 1
                        pbar.update(1)

Grid searching: 100%|██████████| 64/64 [26:29<00:00, 24.26s/it]

In [26]:
df = pd.DataFrame(result).T
df.columns = ["dMethod", "Learning Rate", "Epoch", "Epsilon", "Beta", "Decay", "Akurasi"]
dfs = df.reset_index(drop="index")

In [27]:
dfs

,dMethod,Learning Rate,Epoch,Epsilon,Beta,Decay,Akurasi
0,cosine,0.25,2,0.25,0.25,0.15,0.487978
1,cosine,0.25,2,0.25,0.25,0.25,0.472424
2,cosine,0.25,2,0.25,0.3,0.15,0.487978
3,cosine,0.25,2,0.25,0.3,0.25,0.472424
4,cosine,0.25,2,0.35,0.25,0.15,0.524416
...,...,...,...,...,...,...,...
59,euclidean,0.35,3,0.25,0.3,0.25,0.350608
60,euclidean,0.35,3,0.35,0.25,0.15,0.322159
61,euclidean,0.35,3,0.35,0.25,0.25,0.322159
62,euclidean,0.35,3,0.35,0.3,0.15,0.322159


In [28]:
# dfs.to_csv("../data/processed/comb_evaluation.csv")

In [ ]:
# pd.read_csv()

In [6]:
from pprint import pprint
pprint(result)

{'Combination_1': ['cosine', 0.25, 2, 0.25, 0.25, 0.15, 0.4879778847760378],
 'Combination_10': ['cosine', 0.25, 3, 0.25, 0.25, 0.25, 0.5039567416498322],
 'Combination_11': ['cosine', 0.25, 3, 0.25, 0.3, 0.15, 0.5056579127238483],
 'Combination_12': ['cosine', 0.25, 3, 0.25, 0.3, 0.25, 0.5039567416498322],
 'Combination_13': ['cosine', 0.25, 3, 0.35, 0.25, 0.15, 0.559153667390677],
 'Combination_14': ['cosine', 0.25, 3, 0.35, 0.25, 0.25, 0.5586524294849401],
 'Combination_15': ['cosine', 0.25, 3, 0.35, 0.3, 0.15, 0.559153667390677],
 'Combination_16': ['cosine', 0.25, 3, 0.35, 0.3, 0.25, 0.5586524294849401],
 'Combination_17': ['cosine', 0.35, 2, 0.25, 0.25, 0.15, 0.5358233212327415],
 'Combination_18': ['cosine', 0.35, 2, 0.25, 0.25, 0.25, 0.5540653431960751],
 'Combination_19': ['cosine', 0.35, 2, 0.25, 0.3, 0.15, 0.5358233212327415],
 'Combination_2': ['cosine', 0.25, 2, 0.25, 0.25, 0.25, 0.47242432067074747],
 'Combination_20': ['cosine', 0.35, 2, 0.25, 0.3, 0.25, 0.55406534319607

In [7]:
print(result)

{'Combination_1': ['cosine', 0.25, 2, 0.25, 0.25, 0.15, 0.4879778847760378], 'Combination_2': ['cosine', 0.25, 2, 0.25, 0.25, 0.25, 0.47242432067074747], 'Combination_3': ['cosine', 0.25, 2, 0.25, 0.3, 0.15, 0.4879778847760378], 'Combination_4': ['cosine', 0.25, 2, 0.25, 0.3, 0.25, 0.47242432067074747], 'Combination_5': ['cosine', 0.25, 2, 0.35, 0.25, 0.15, 0.5244163616203654], 'Combination_6': ['cosine', 0.25, 2, 0.35, 0.25, 0.25, 0.5334538329510762], 'Combination_7': ['cosine', 0.25, 2, 0.35, 0.3, 0.15, 0.5244163616203654], 'Combination_8': ['cosine', 0.25, 2, 0.35, 0.3, 0.25, 0.5334538329510762], 'Combination_9': ['cosine', 0.25, 3, 0.25, 0.25, 0.15, 0.5056579127238483], 'Combination_10': ['cosine', 0.25, 3, 0.25, 0.25, 0.25, 0.5039567416498322], 'Combination_11': ['cosine', 0.25, 3, 0.25, 0.3, 0.15, 0.5056579127238483], 'Combination_12': ['cosine', 0.25, 3, 0.25, 0.3, 0.25, 0.5039567416498322], 'Combination_13': ['cosine', 0.25, 3, 0.35, 0.25, 0.15, 0.559153667390677], 'Combination

In [8]:
output_size = np.unique(y_l).shape[0]
input_size = X_train.shape[1]
sfs = LVQ3(input_size=input_size, output_size=output_size, random_state=201, epoch=3, alpha=0.45, beta=0.25, m=0.2,
           epsilon=0.25, dMethod="cosine")
sfs.fit(X_train, y_train)


In [9]:
res = sfs.predict(X_test)
print(classification_report(res, y_test, zero_division=1))


              precision    recall  f1-score   support

           0       0.65      0.77      0.71     19430
           1       0.23      0.44      0.30     11326
           2       0.78      0.48      0.59     35081

    accuracy                           0.56     65837
   macro avg       0.56      0.56      0.54     65837
weighted avg       0.65      0.56      0.58     65837
